<a href="https://colab.research.google.com/github/rachit-shah/ADBI-projects/blob/master/Speaker_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [36]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=2bdfbaa22aae8825076d5fce4a52321361f27be5b52783b7d81a5f2a452dc659
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import os
import glob
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import keras.backend as K
import keras
from python_speech_features import fbank, delta
import operator
from collections import OrderedDict
from multiprocessing import Pool
from time import time

In [0]:
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partaa .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partab .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partac .
!cp drive/My\ Drive/voxceleb1/vox1_dev_wav_partad .

In [0]:
!cat vox1_dev* > vox1_dev_wav.zip

In [0]:
!cp vox1_dev_wav.zip drive/My\ Drive

In [0]:
!rm vox1_dev_wav_partaa
!rm vox1_dev_wav_partab
!rm vox1_dev_wav_partac
!rm vox1_dev_wav_partad


In [0]:
!unzip vox1_dev_wav.zip

In [22]:
!ls wav/id10001/eWIX7sfn-M0

00001.wav


In [23]:
import IPython.display as ipd
ipd.Audio('wav/id10001/eWIX7sfn-M0/00001.wav') # load a local WAV file

In [0]:
sample_rate = 12000

In [0]:
def create_files_dictionary():
  train_path = "wav"
  pattern = "**/*.wav"
  files = glob.glob(os.path.join(train_path, pattern), recursive=True)

  classes = []
  for file in files:
    filename = file.split("/")[-1]
    classes.append(file.split("/")[1])
    
  spk_uniq = np.unique(classes)

  train_dict = defaultdict(list)
  print("Number of Classes" ,len(spk_uniq))

  for i in range(len(classes)):
    train_dict[classes[i]].append(files[i])

  return train_dict,spk_uniq

In [39]:
train_dict, spk_uniq = create_files_dictionary()

Number of Classes 1211


In [0]:
subset_train_dict = OrderedDict(sorted(train_dict.items(), key=lambda x: len(x[1]),reverse=True)[:300])

In [0]:
[len(subset_train_dict[i]) for i in subset_train_dict]

In [0]:
import json
with open('subset_files_300.json','w') as f:
  json.dump(subset_train_dict,f,indent=4)

In [0]:
with open('subset_files_300.json','r') as f:
  subset_train_dict = json.load(f)

In [0]:
!mkdir subset2

In [77]:
!ls

a.txt  drive  sample_data  subset  subset_files_300.json  vox1_dev_wav.zip  wav


In [0]:
from shutil import copyfile

In [0]:
for classid, files in subset_train_dict.items():
  os.mkdir("subset2/"+classid)
  i=0
  for filepath in files:
    filename = filepath.split("/")[-1]
    copyfile(filepath,"subset2/"+classid+"/"+str(i)+".wav")
    i+=1
    if i>=150:
      break


In [0]:
!zip -rq subset2.zip subset2

In [0]:
!cp subset2.zip drive/My\ Drive

In [98]:
!ls -l

total 59819436
-rw-r--r--    1 root root          14 Nov 28 21:09 a.txt
drwx------    3 root root        4096 Nov 28 20:44 drive
drwxr-xr-x    1 root root        4096 Nov 21 16:30 sample_data
drwxr-xr-x  302 root root       12288 Nov 28 23:06 subset
drwxr-xr-x  302 root root       12288 Nov 28 23:34 subset2
-rw-r--r--    1 root root 10021040192 Nov 28 23:49 subset2.zip
-rw-r--r--    1 root root     3072932 Nov 28 22:38 subset_files_300.json
-rw-r--r--    1 root root 18634264892 Nov 28 23:25 subset.zip
-rw-r--r--    1 root root 32596624183 Nov 28 21:04 vox1_dev_wav.zip
drwxrwxr-x 1213 root root       36864 May 29  2018 wav


In [0]:
def read_audio(filename,sample_rate):
  audio, _ = librosa.load(filename, sr=sample_rate, mono=True)
  return audio.flatten()

In [0]:
read_audio('subset2/id10108/22.wav',12000).shape

(68641,)

In [107]:
import IPython.display as ipd
ipd.Audio('subset2/id10108/22.wav') # load a local WAV file

In [0]:
def load_audio(filename, cnn=True):
  audio = read_audio(filename,sample_rate)
  start_sec, end_sec = 1,5
  start_frame = int(start_sec * sample_rate)
  end_frame = int(end_sec * sample_rate)
  audio = audio[start_frame:end_frame]

  #Padding with zeros if audio length less than 4 secs
  if len(audio)<(end_frame-start_frame):
    au = [0]*(end_frame-start_frame)
    for i in range(len(audio)):
        au[i] = audio[i]
    audio = np.array(au)
  #print(len(audio),sample_rate)

  filter_banks, energies = fbank(audio, samplerate=sample_rate, nfilt=64, winlen=0.025) #filter_bank (num_frames , 64),energies (num_frames ,)
  #Standardize with mean and std
  filter_banks = [(i - np.mean(i)) / np.std(i) for i in filter_banks]
  #print("filterbanks:" + str(len(filter_banks)))
  frames_features = filter_banks

  num_frames = len(frames_features)
  #print(num_frames)
  """
  network_inputs = []
  if cnn:
    for j in range(24, num_frames - 24):
      frames_slice = frames_features[j - 24:j + 24]
      print(np.array(frames_slice).shape)
      network_inputs.append(np.reshape(np.array(frames_slice), (32,32, 3)))
    #print(np.array(frames_slice).shape)
    final = np.array(network_inputs)
    #print(final.shape)
    return final
  
  else:
    for j in range(6, num_frames - 6):
      frames_slice = frames_features[j - 6:j + 6]
      #print(np.array(frames_slice).shape)
      network_inputs.append(np.reshape(np.array(frames_slice), (16,16, 3)))
    #print(np.array(frames_slice).shape)
    final = np.array(network_inputs)
    #print(final.shape)
    return final
  """
  return np.reshape(np.array(frames_features),(num_frames, 64, 1))



In [141]:
load_audio('subset2/id10108/22.wav').shape

(399, 64, 1)

In [0]:
def create_catalogue(path,pattern="*/*.wav"):
  data = pd.DataFrame()
  data['filename'] = glob.glob(os.path.join(path, pattern), recursive=True)
  data['class'] = data['filename'].apply(lambda x: x.split("/")[1])
  num_speakers = len(data['class'].unique())
  print('Found {} files with {} different speakers.'.format(str(len(data)), str(num_speakers)))
  # print(data.head(10))
  return data

In [127]:
catalog = create_catalogue("subset2")

Found 44998 files with 300 different speakers.


In [131]:
catalog

,filename,class
0,subset2/id10225/73.wav,id10225
1,subset2/id10225/143.wav,id10225
2,subset2/id10225/29.wav,id10225
3,subset2/id10225/23.wav,id10225
4,subset2/id10225/104.wav,id10225
...,...,...
44993,subset2/id10108/128.wav,id10108
44994,subset2/id10108/142.wav,id10108
44995,subset2/id10108/6.wav,id10108
44996,subset2/id10108/109.wav,id10108


In [133]:
catalog.iloc[0]['filename']

'subset2/id10225/73.wav'

In [0]:
!mkdir subset2_npy

In [0]:
def preprocess_batch(data,out_dir="subset2_npy",task_num="0"):
  start_time = time()
  i=0
  for i in range(len(data)):
    orig_time = time()
    filename = data.iloc[i]['filename']
    target_filename = out_dir+"/" + filename.split("/")[1]+"_"+filename.split("/")[-1].split('.')[0] + '.npy'
    #print('!!!',target_filename)
    if os.path.exists(target_filename):
      if i % 10 == 0: print("task:{0} No.:{1} Exist File:{2}".format(task_num, i, filename))
      continue
    feature = load_audio(filename)
    #print('!!!',feature.ndim,feature.shape[0],feature.shape[1],feature.shape[2])
    if feature.ndim != 3 or feature.shape[0] < 399 or feature.shape[1] !=64 or feature.shape[2] != 1:
      print('there is an error in file:',filename)
      continue
    np.save(target_filename, feature)
    if i % 100 == 0:
      print("task:{0} cost time per audio: {1:.3f}s No.:{2} File name:{3}".format(task_num, time() - orig_time, i, filename))
  print("task %s runs %d seconds. %d files" %(task_num, time()-start_time,i))

In [0]:
def preprocess(data_dir,out_dir):
  orig_time = time()
  data = create_catalogue(data_dir)
  print("extract fbank from audio and save as npy, using multiprocessing pool........ ")
  p = Pool(5)
  print(len(data))
  patch = int(len(data)/5)
  for i in range(5):
      if i < 4:
          sdata=data[i*patch: (i+1)*patch]
      else:
          sdata = data[i*patch:]
      print("task %s sdata length: %d" %(i, len(sdata)))
      p.apply_async(preprocess_batch, args=(sdata,out_dir,i))
  print('Waiting for all subprocesses done...')
  p.close()
  p.join()

  print("Extract audio features and save it as npy file, cost {0} seconds".format(time()-orig_time))

In [172]:
preprocess("subset2","subset2_npy")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

task:0 cost time per audio: 0.951s No.:3400 File name:subset2/id10059/100.wav


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

task:2 cost time per audio: 0.886s No.:3300 File name:subset2/id10147/73.wav


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

...

task 1 runs 9465 seconds. 8998 files


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

task:2 cost time per audio: 0.429s No.:8900 File name:subset2/id10111/41.wav


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

task 2 runs 9510 seconds. 8998 files
Extract audio features and save it as npy file, cost 9510.675946474075 seconds


In [173]:
len(os.listdir("subset2_npy"))

44998

In [0]:
!zip -rq subset2_npy.zip subset2_npy
!cp subset2_npy.zip drive/My\ Drive

In [175]:
!ls -l

total 68090680
-rw-r--r--    1 root root          14 Nov 28 21:09 a.txt
drwx------    3 root root        4096 Nov 28 20:44 drive
drwxr-xr-x    1 root root        4096 Nov 21 16:30 sample_data
drwxr-xr-x  302 root root       12288 Nov 28 23:06 subset
drwxr-xr-x  302 root root       12288 Nov 28 23:34 subset2
drwxr-xr-x    2 root root     1622016 Nov 29 03:28 subset2_npy
-rw-r--r--    1 root root  8468125037 Nov 29 03:37 subset2_npy.zip
-rw-r--r--    1 root root 10021040192 Nov 28 23:49 subset2.zip
-rw-r--r--    1 root root     3072932 Nov 28 22:38 subset_files_300.json
-rw-r--r--    1 root root 18634264892 Nov 28 23:25 subset.zip
-rw-r--r--    1 root root 32596624183 Nov 28 21:04 vox1_dev_wav.zip
drwxrwxr-x 1213 root root       36864 May 29  2018 wav


In [0]:
count=0
for file in os.listdir('subset2_npy'):
  if np.isnan(np.load('subset2_npy/'+file)).all():
    print(file)

In [187]:
np.load('subset2_npy/id10105_49.npy')[:200]

array([[[-0.48573157],
        [-0.21166633],
        [-0.44857382],
        ...,
        [-0.5380707 ],
        [-0.55956972],
        [-0.59433054]],

       [[-0.58372998],
        [-0.44701983],
        [-0.49327522],
        ...,
        [-0.56590745],
        [-0.57750348],
        [-0.57892441]],

       [[-0.3729707 ],
        [-0.37675034],
        [-0.38155315],
        ...,
        [-0.38457981],
        [-0.38809764],
        [-0.38746263]],

       ...,

       [[ 1.45953741],
        [ 5.51799197],
        [ 0.99107831],
        ...,
        [-0.3562527 ],
        [-0.39604435],
        [-0.77592862]],

       [[ 1.2011199 ],
        [ 4.09617843],
        [ 1.01663784],
        ...,
        [ 0.1184639 ],
        [-0.34126484],
        [-1.13166228]],

       [[ 0.67083138],
        [-0.53738105],
        [ 2.73042956],
        ...,
        [ 0.48847156],
        [-0.4722995 ],
        [-1.04171265]]])

FileNotFoundError: ignored

In [0]:
NUM_FRAMES=399
BATCH_SIZE=32

In [0]:
def clipped_audio(x, num_frames=NUM_FRAMES):
    if x.shape[0] > num_frames:
        bias = np.random.randint(0, x.shape[0] - num_frames)
        clipped_x = x[bias: num_frames + bias]
    else:
        clipped_x = x

    return clipped_x


class MiniBatch:
    def __init__(self, data, batch_size, unique_speakers=None):    
        if unique_speakers is None:
            unique_speakers = list(data['class'].unique())
        num_triplets = batch_size

        anchor_batch = None
        positive_batch = None
        negative_batch = None
        for ii in range(num_triplets):
            two_different_speakers = np.random.choice(unique_speakers, size=2, replace=False)
            anchor_positive_speaker = two_different_speakers[0]
            negative_speaker = two_different_speakers[1]
            anchor_positive_file = data[data['class'] == anchor_positive_speaker].sample(n=2, replace=False)
            anchor_df = pd.DataFrame(anchor_positive_file[0:1])
            anchor_df['training_type'] = 'anchor'
            positive_df = pd.DataFrame(anchor_positive_file[1:2])
            positive_df['training_type'] = 'positive'
            negative_df = data[data['class'] == negative_speaker].sample(n=1)
            negative_df['training_type'] = 'negative'

            if anchor_batch is None:
                anchor_batch = anchor_df.copy()
            else:
                anchor_batch = pd.concat([anchor_batch, anchor_df], axis=0)
            if positive_batch is None:
                positive_batch = positive_df.copy()
            else:
                positive_batch = pd.concat([positive_batch, positive_df], axis=0)
            if negative_batch is None:
                negative_batch = negative_df.copy()
            else:
                negative_batch = pd.concat([negative_batch, negative_df], axis=0)

        self.data_batch = pd.DataFrame(pd.concat([anchor_batch, positive_batch, negative_batch], axis=0))
        self.num_triplets = num_triplets

    def to_inputs(self):

        new_x = []
        for i in range(len(self.data_batch)):
            filename = self.data_batch.iloc[i]['filename']
            x = np.load(filename)
            new_x.append(clipped_audio(x))
        x = np.array(new_x) #(batchsize, num_frames, 64, 1)
        y = self.data_batch['class'].values

        # anchor examples [speakers] == positive examples [speakers]
        np.testing.assert_array_equal(y[0:self.num_triplets], y[self.num_triplets:2 * self.num_triplets])
        return x, y


def stochastic_mini_batch(data, batch_size=BATCH_SIZE,unique_speakers=None):
    mini_batch = MiniBatch(data, batch_size,unique_speakers)
    return mini_batch